In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [10]:
feature_list = ['x','y','w0','w1','w2','w3','w4','w5','w6','day','month','year', 'accuracy']
for i in range(0,24):
    feature_list.append('h' + str(i))

In [11]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    #df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    for i in range(0,24):
        df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24) * fw[2]
    
    df['w0'] = ((d_times.weekday + 0) % 7) * fw[3]
    df['w1'] = ((d_times.weekday + 1) % 7) * fw[3]
    df['w2'] = ((d_times.weekday + 2) % 7) * fw[3]
    df['w3'] = ((d_times.weekday + 3) % 7) * fw[3]
    df['w4'] = ((d_times.weekday + 4) % 7) * fw[3]
    df['w5'] = ((d_times.weekday + 5) % 7) * fw[3]
    df['w6'] = ((d_times.weekday + 6) % 7) * fw[3]
    
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    #df.accuracy = df.accuracy.values * fw[7]
    df['accuracy'] = np.log10(df.accuracy) * fw[7]
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30)) * fw[8]
    df = df.drop(['time'], axis=1)
    
    return df

In [12]:
def calculate_distance(distances):
    return distances ** -2

In [13]:
def process_one_cell(df_train, df_test, th):    
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    row_ids = df_test.row_id
    
    best_k=np.floor(np.sqrt(len(df_train.index)/4*best_mul)/5)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=best_k.astype(int), weights=calculate_distance, 
                               metric='manhattan')
    clf.fit(df_train[feature_list], df_train.place_id)
    predictions = clf.predict_proba(df_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [14]:
def run_prediction(df_train, df_test):
    df_train = prepare_data(df_train)
    df_test = prepare_data(df_test)
    df_test['p1'] = np.nan
    df_test['p2'] = np.nan
    df_test['p3'] = np.nan
    
    prediction_result = process_one_cell(df_train, df_test, 1)
    prediction_result.sort_index(inplace=True)
    return prediction_result

In [15]:
def run_validation():
    df = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    final_validation_result = pd.DataFrame()
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.time <= 786239 * 0.875) & \
                               (df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_validation_cell = df[(df.time > 786239 * 0.875) & \
                                    (df.x >= min_x) & \
                                    (df.x < max_x) & \
                                    (df.y >= min_y) & \
                                    (df.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_validation_cell)
            final_validation_result = final_validation_result.append(prediction_result)
            # Calculate score
            prediction_result.sort_index(inplace=True)
            prediction_result['score'] = (prediction_result.p1 == df_validation_cell.place_id) * 1
            prediction_result['score'] += (prediction_result.p2 == df_validation_cell.place_id) * 0.5
            prediction_result['score'] += (prediction_result.p3 == df_validation_cell.place_id) * 0.33

            score = prediction_result.score.mean()
            print('s1:', score, flush=True)
            total_score += score
            score_count += 1
            
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
    print("Final:", total_score/score_count, flush=True)
    final_validation_result.sort_index(inplace=True)
    final_validation_result.to_csv('Base620ValidationResult.csv', index=False)


In [16]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_validation()

s1: 0.5516139590240542
s1: 0.5763827671046391
s1: 0.5711775178026414
s1: 0.5699491937265259
s1: 0.5451112028058555
s1: 0.5515186358099845
s1: 0.5615268827320152
s1: 0.5723161097207871
s1: 0.5793890717577316
s1: 0.57972467677425
s1: 0.5742619058924038
s1: 0.585577287609853
s1: 0.5779490686158889
s1: 0.5955150214592237
s1: 0.5808755102040781
s1: 0.5999383496680332
s1: 0.5696937350359098
s1: 0.6068837099811638
s1: 0.5855698992443286
s1: 0.5720725515885972
Elapsed time overall: 548.9417042732239 seconds 0
s1: 0.5604248064872797
s1: 0.5526919602529322
s1: 0.5409229194771319
s1: 0.5558043893129734
s1: 0.5145208351533114
s1: 0.5548714352501126
s1: 0.534155590491159
s1: 0.5469139177168233
s1: 0.5029728289824151
s1: 0.5215326184092898
s1: 0.5464584372402291
s1: 0.5566686998545454
s1: 0.5378697342493436
s1: 0.5137105129076047
s1: 0.5394763606377091
s1: 0.5196218258295646
s1: 0.5598121471676041
s1: 0.5515205059053246
s1: 0.5146341463414593
s1: 0.532567182035704
Elapsed time overall: 593.868444204

In [ ]:
def run_test():
    
    # Run test
    df = pd.read_csv('../../train.csv',
                           usecols=['row_id','x','y','accuracy','time','place_id'])
    df_test = pd.read_csv('../../test.csv',
                           usecols=['row_id','x','y','accuracy','time'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    total_result = pd.DataFrame()
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_test_cell = df_test[(df_test.x >= min_x) & \
                                   (df_test.x < max_x) & \
                                   (df_test.y >= min_y) & \
                                   (df_test.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_test_cell)
            total_result = total_result.append(prediction_result)
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
            
    
    total_result.sort_index(inplace=True)
    total_result['place_id'] = total_result.p1.astype(str) + " " + \
                               total_result.p2.astype(str) + " " + \
                               total_result.p3.astype(str)
    total_result[['row_id', 'place_id']].to_csv('Baseline620.csv', index=False)

In [ ]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 23, 4.5]
best_mul = 0.6
run_test()

In [ ]:
0.58065